In [1]:
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

#import openai
#from langchain import *
#from pydantic import *
import os 
import pandas as pd 

with open("../secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1]


In [3]:


example_formatter = """sentence: {sentence}
Tag: {tag}"""

example_prompt = PromptTemplate(
    input_variables=["sentence", "tag"],
    template=example_formatter
)

suffix = "sentence: {sentence}\nTag:"

body_prefix = """Classify the following as one of:

1. body: if it mention of body parts such as face, head, back, shoulder, feet, leg, etc.
2. not body: if the description does not mention any body parts """

position_prefix = """Classify the following as one of:

1. position: if the description includes positional language such as right, left, up, over, etc
2. not position: if the description does not mention positions or facing directions """

shape_prefix = """Classify the following as one of:

1. shape: if the description includes a geometric shape word (ex: shape, square, diamond, triangle, angle, point, etc)
2. not shape: if the description does not reference geometric shapes"""

posture_prefix = """Classify the following as one of:

1. posture: if the description is about the posture or action of a person or creature (ex: standing, marching, kicking, etc)
2. not posture: if the description is not about a person or creature's posture"""


def make_chat_template(instruction):
    system_message = SystemMessage(content=instruction)
    messages = []
    messages.append(HumanMessagePromptTemplate.from_template("{test_sentence}"))

    return ChatPromptTemplate.from_messages([system_message, *messages])

body_chat = make_chat_template(body_prefix)
position_chat = make_chat_template(position_prefix)
shape_chat = make_chat_template(shape_prefix)
posture_chat = make_chat_template(posture_prefix)



    

In [4]:



def tag_data_with_langchain(df, model, prompt): 
  
  res = []
  #for index in range(100):
  for index, _ in df.iterrows():
    print(index)
    #print(model)
    #print(prompt)
    chain=LLMChain(llm=model,prompt=prompt)
    output=chain.run(df["chunk"][index])
    print(df["chunk"][index])
    #string_output=output.content
    print(output)
    res.append(output)
  return res





In [5]:
model_name = "gpt-3.5-turbo"
#model_name="gpt-4"
temperature = 0
llm = ChatOpenAI(model_name=model_name, max_retries=15, temperature=0)

chain=LLMChain
input =pd.read_csv("../data/test/pre_tag/4pB.csv")

test_input=input
#print(test_input)

body = tag_data_with_langchain(test_input, llm, body_chat)
position = tag_data_with_langchain(test_input, llm, position_chat)
shape= tag_data_with_langchain(test_input, llm, shape_chat)
posture=tag_data_with_langchain(test_input, llm, posture_chat)

output=test_input
output['body']=body
output['position']=position
output['shape']=shape
output['posture']=posture

output.to_csv("../data/test/post_tag/4pB.csv")


0
sitting with his arms out
not body
1
the two triangles go out either side just below the head
not body
2
the triangle on the right comes away from the "body" a bit
not body
3
one foot out to the left
body
4
a lady facing left
not body
5
her triangle arms
1. body
6
face left
1. body
7
a triangle behind her neck
not body
8
facing left
not body
9
kneeling
1. body
10
a woman in a dress
not body
11
with her left foot out
1. body
12
both arms are raised to each side
body
13
head tilted to the right and down slightly
body
14
right arm is away from her body slightly
body
15
one triangle on the floor
not body
16
to the left
not body
17
triangle at the bottom of her dress thing facing left
not body
18
one of the tricky ones
not body
19
kneeling to the left
not body
20
triangle arms out wide
not body
21
right arm detached from body
body
22
kneeling to the right
not body
23
falling to the right
not body
24
a dog facing left
not body
25
with a square and triangle for ears
not body
26
one triangle

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


two triangles
not body
40
wings
not body
41
wings
not body
42
man kneeling
not body
43
head facing down
1. body
44
wings
not body
45
man head facing down
1. body
46


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 15 Nov 2023 19:57:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '826a0c8f39fd1567-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


kneeling
1. body
47
wings
not body
48
kneeling
1. body
49
head tilted down
body
50
mouse
not body
51
mouse
not body
52
bunny ears
not body
53
point of square
not body
54
point of 2 different triangles touches
not body
55
the guy with wings
not body
56
wings
not body
57
a flat bottom part
not body
58
two small triangles on each side
not body
59
kneeling person
body
60
feet pointing to the left
body
61
backwards c shape
not body
62
on the left edge
not body
63
the bottom right has a right angle
not body
64
triangles on the left and right
not body
65
the right triangle is separated from the body
not body
66
the left triangle is connected
not body
67
right triangle separated from body
not body
68
left triangle connected
not body
69
disjointed triangle to the right
not body
70
joined to the left
not body
71
right triangle
not body
72
not joined
not body
73
left one joined
not body
74
disconnected triangle on right
not body
75
kneeling with their hands stretched out
not body
76
not the one f

...

two triangles pointing left and right
not posture
361
smaller triangle on the bottom left
not posture
362
looking left
not posture
363
diamond head
not posture
364
sitting down
1. posture
365
diamond on top
not posture
366
two triangles right below facing each direction
not posture
367
big blob on bottom
not posture
368
diamond on top
not posture
369
two triangles on both sides
not posture
370
another smaller triangle on the bottom left side
not posture
371
square
not posture
372
balancing on the point
posture
373
two triangles
not posture
374


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 16 Nov 2023 21:36:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '8272d092bc3422ba-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


touching
not posture
375
kneeling figure
1. posture
376
under the square
not posture
377
a smaller triangle on the right
not posture
378
a larger one on the left
not posture
379
arms out
1. posture
380
arms facing slightly down
1. posture
381
arms out
1. posture
382
small triangle
not posture
383
bottom left
not posture
384
the square
not posture
385
balancing on top of 2 triangles
posture
386
kneeling
1. posture
387
one triangle is bigger
not posture
388
the other (on the right) is smaller
not posture
389
on knees
1. posture
390
square is balancing on two triangles
not posture
391
third triangle
not posture
392
bottom left
not posture
393
the one triangle on the right is tilted
not posture
394
facing right or left
posture
395
right
not posture
396
the triangle at the bottom left
not posture
397
the feet
not posture
398
a dog begging for food
posture
399
facing left
posture
400
square balancing on the corner
not posture
401
triangle right below it on the right
not posture
402
ears
not 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


arms out each side
1. posture
484
one foot on floor
posture
485
the other his knee
not posture
486
man on his knees
1. posture
487
both arms stretched out both sides
1. posture
488
part of his leg is shown
not posture
489
with his feet
1. posture
490
gymnast
1. posture
491
on one knee
1. posture
492
arms straight out
1. posture
493
head slightly tilted down
posture
494
nun kneeling at church
1. posture
495
arms stretched out in front of her
1. posture
496
one arm stretched out front
posture
497
head down
1. posture
498
straight
1. posture
499
the other arm behind her
1. posture
500
head slightly down
posture
501
knees facing right
posture
502
facing right
not posture
503
in a kneeling position
1. posture
504
both arms outstretched
1. posture
505
similar to a soccer player celebrating a goal
1. posture
506
head is tilted slightly downwards
posture
507
victory celebration after scoring a goal
not posture
508
facing right
not posture
509
imagine being in a lunge position for exercising
1.